# Toxic comment analysis

---


## Load libraries


In [120]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [121]:
import pandas as pd
import numpy as np
import nltk

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/louislecouturier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/louislecouturier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load Dataset


In [122]:
data = pd.read_csv("datasets/train.csv")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


### Clean the data


In [123]:
print("Number of missing values in each column :")
print(data.isnull().sum())

Number of missing values in each column :
id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


We will remove the rows with missing values and remove the id column as it is not relevant.


In [124]:
data = data.dropna()
del data["id"]

Our data is now clean and ready for analysis !


In [125]:
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [126]:
data[data["obscene"] == 1]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
51,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK T...,1,0,1,0,0,0
55,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
...,...,...,...,...,...,...,...
159411,Fat piece of shit \n\nyou obese piece of shit....,1,0,1,0,1,0
159493,FUCKING FAGGOT \n\nLOLWAT.,1,0,1,0,1,0
159494,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159541,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0


### Loose analysis data

We will first remove the `toxic`, `severe_toxic`, `obscene`, `threat`, `insult`, `identity_hate` columns to simplify the problem.


In [127]:
data.drop(
    ["severe_toxic", "obscene", "threat", "insult", "identity_hate"],
    axis=1,
    inplace=True,
)
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


## Reduce the size of the dataset

In [128]:
data = data.head(15000)

### Clean the comments


In [129]:
data["comment_text"].head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [130]:
from helpers.data.text_manipulation import TextManipulation

text_manipulator = TextManipulation()

comments = data["comment_text"]
labels = data["toxic"]

comments = comments.map(lambda x: text_manipulator.remove_contractions(x))
comments = comments.map(lambda x: text_manipulator.remove_abbreviations(x))
comments = comments.map(lambda x: text_manipulator.remove_punctuation(x))

comments

0        Explanation Why the edits made under my userna...
1        Daww He matches this background colour I am se...
2        Hey man I am really not trying to edit war It ...
3         More I cannot make any real suggestions on im...
4        You sir are my hero Any chance you remember wh...
                               ...                        
14995     Women getting killed in a domestic homicide i...
14996                   Keep up the good work 172201253232
14997     Sounds great talk • contribs Use reJc86035 to...
14998     OkGA Early I know but I do not want to forget...
14999     Agni V accuracy Here irondome is questioning ...
Name: comment_text, Length: 15000, dtype: object

### Tokenize the comments

In [131]:
tokens = text_manipulator.tokenize(comments)

In [132]:
print(tokens)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Remove the stopwords

In [133]:
stop_words = nltk.corpus.stopwords.words("english")

In [134]:
# Remove the stop words
tokens = [text_manipulator.remove_stopwords(x, stop_words) for x in tokens]

tokens

[['Explanation',
  'Why',
  'edits',
  'made',
  'username',
  'Hardcore',
  'Metallica',
  'Fan',
  'reverted',
  'They',
  'vandalisms',
  'closure',
  'GAs',
  'I',
  'voted',
  'New',
  'York',
  'Dolls',
  'FAC',
  'And',
  'please',
  'remove',
  'template',
  'talk',
  'page',
  'since',
  'I',
  'retired',
  'now892053827'],
 ['Daww',
  'He',
  'matches',
  'background',
  'colour',
  'I',
  'seemingly',
  'stuck',
  'Thanks',
  'talk',
  '2151',
  'January',
  '11',
  '2016',
  'UTC'],
 ['Hey',
  'man',
  'I',
  'really',
  'trying',
  'edit',
  'war',
  'It',
  'guy',
  'constantly',
  'removing',
  'relevant',
  'information',
  'talking',
  'edits',
  'instead',
  'talk',
  'page',
  'He',
  'seems',
  'care',
  'formatting',
  'actual',
  'info'],
 ['More',
  'I',
  'make',
  'real',
  'suggestions',
  'improvement',
  'I',
  'wondered',
  'section',
  'statistics',
  'later',
  'subsection',
  'types',
  'accidents',
  'I',
  'think',
  'references',
  'may',
  'need',
  

## Analyse the text

### TF_IDF

In [135]:
from sklearn.model_selection import train_test_split



tokens = [" ".join(x) for x in tokens]

# 


In [136]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)

In [137]:
MODEL_PATH = "models/TF-IDF/model.pkl"
VECTORIZER_PATH = "models/TF-IDF/vectorizer.pkl"

In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pickle


tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(tokens)

pickle.dump(tfidf, open(VECTORIZER_PATH, "wb+"))


text_classifier = RandomForestClassifier(15)
text_classifier.fit(X_train, y_train)
pickle.dump(text_classifier, open(MODEL_PATH, "wb+"))

In [139]:
predictions = text_classifier.predict(X_test)

In [140]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(f"confusion matrix : \n{confusion_matrix(y_test, predictions)}\n")
print(f"classification report : {classification_report(y_test, predictions)}\n")
print(f"accuracy : {accuracy_score(y_test, predictions)}")

confusion matrix : 
[[2667   38]
 [ 150  145]]

classification report :               precision    recall  f1-score   support

           0       0.95      0.99      0.97      2705
           1       0.79      0.49      0.61       295

    accuracy                           0.94      3000
   macro avg       0.87      0.74      0.79      3000
weighted avg       0.93      0.94      0.93      3000


accuracy : 0.9373333333333334
